In [ ]:
pip install aidkit

In [ ]:
aidkit --uri https://github.com/example/repo.git

In [ ]:
from aidkits import MarkdownCrawler

MarkdownCrawler("repo/path").work()

In [ ]:
aidkit --uri ./local_directory --output_path result.json

In [ ]:
[
  {
    "title": "example.md",
    "chunks": [
      {
        "title": "Header 1",
        "content": "Content under Header 1",
        "length": 120,
        "chunk_num": 1,
        "chunk_amount": 2
      },
      {
        "title": "Header 2",
        "content": "Content under Header 2",
        "length": 240,
        "chunk_num": 2,
        "chunk_amount": 2
      }
    ]
  }
]

In [ ]:
from opensearchpy import OpenSearch
from sentence_transformers import SentenceTransformer
from aidkits.opensearch_retriever import OpenSearchRetriever
from aidkits.models import LibrarySource

# Initialize the OpenSearch client
client = OpenSearch(
   hosts=[{"host": "localhost", "port": 9200}],
   http_auth=("admin", "admin"),
   use_ssl=False,
   verify_certs=False,
)

# Initialize the encoder
encoder = SentenceTransformer("all-MiniLM-L6-v2")

# Create the retriever
retriever = OpenSearchRetriever(client, encoder)

# Create a collection
retriever.create_collection("documentation")

# Upload a library
library = LibrarySource.from_json("path/to/library.json")
retriever.upload_library(library)

# Search for documents
results = retriever.search(
   question="How do I use the API?",
   collection_name="documentation",
   top_k=5
)

# Print the results
for result in results:
   print(result.markdown)

In [ ]:
from langchain_core.language_models import ChatOpenAI
from aidkits.documentation_tool import DocumentationTool
from aidkits.opensearch_retriever import OpenSearchRetriever

# Initialize the language model
llm = ChatOpenAI(model="gpt-3.5-turbo")

# Initialize the retriever (as shown above)
# ...

# Create the documentation tool
doc_tool = DocumentationTool(
   llm=llm,
   retriever=retriever,
   collection_name="documentation",
   top_k=5
)

# Answer a question
answer = doc_tool.invoke({"question": "How do I use the API?"})
print(answer)